<a href="https://colab.research.google.com/github/aneesh2711/Bioinformatics/blob/master/Keras_multi_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from numpy import inf
import math

from sklearn.metrics import f1_score

from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data_fn = "/content/drive/My Drive/Colab Notebooks/Protien_prediction/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
branch_term = "GO0071840"

In [4]:
shared_layers_num = 2
spec_layers_num = 2
shared_layers_units = [500,200,100]
spec_layers_units = [100,100,100]
dropout = [0.4,0.3,0.3]
optimizer = 'adam'
metrics=['accuracy']
batch_size = 128
oversample = False

In [5]:
all_data_x_fn = data_fn + '/all_data_X.csv'
all_data_x = pd.read_csv(all_data_x_fn, sep='\t', header=0, index_col=0)
all_proteins_train = [p.replace('"', '') for p in all_data_x.index]
all_data_x.index = all_proteins_train

all_data_y_fn = data_fn + '/all_data_Y.csv'
all_data_y = pd.read_csv(all_data_y_fn, sep='\t', header=0, index_col=0)
all_proteins_train = [p.replace('"', '') for p in all_data_y.index]
all_data_y.index = all_proteins_train

In [6]:
train_fn = data_fn + '/train_sets/' + branch_term + '_train.csv'
train_y = pd.read_csv(train_fn, sep='\t', header=0, index_col=0)
tasks = train_y.columns
proteins_train = [p for p in train_y.index if p in all_data_x.index]
train_x = all_data_x.loc[proteins_train, :].values
train_y = all_data_y.loc[proteins_train, tasks].values
validate_fn = data_fn + '/train_sets/' + branch_term + '_valid.csv'
validate_y = pd.read_csv(validate_fn, sep='\t', header=0, index_col=0)
proteins_validate = [p for p in validate_y.index if p in all_data_x.index]
validate_x = all_data_x.loc[proteins_validate, :].values
validate_y = all_data_y.loc[proteins_validate, tasks].values

In [7]:
loss= ['binary_crossentropy']*len(tasks)

In [8]:
def test():
  train_set = {}
  validate_set = {}
  train_labels = []
  validation_labels = []
  for task in tasks:
    proteins_in_train = train_y.index[train_y.loc[:, task] != inf]
    train_y = train_y.loc[proteins_in_train, :]
    proteins_in_validate = validate_y.index[validate_y.loc[:, task] != inf]
    validate_y = validate_y.loc[proteins_in_validate, :]
  for task in tasks:
    train_labels.append(train_y.loc[proteins_in_train,task].values)
    validation_labels.append(validate_y.loc[proteins_in_validate,task].values)
  train_set = [train_x.loc[proteins_in_train, :].values, train_labels]
  validate_set = [validate_x.loc[proteins_in_validate, :].values, validation_labels]

In [9]:
shared_layers = []
spec_layers = {}
outputs = []

inputs = Input(shape=(258,))
x = Dropout(dropout[0])(inputs)
for i in range(shared_layers_num):
  name = "shared-"+ str(i)
  layer = Dense(shared_layers_units[i], activation='relu',name = name)(x)
  shared_layers.append(layer)
  x = Dropout(dropout[1])(layer)
for task in tasks:
  spec_layers[task] = []
  layer = Dense(spec_layers_units[0], activation='relu',name = task+"-spec-0")(x)
  spec_layers[task].append(layer)
  y = Dropout(dropout[2])(layer)
  for i in range(1,spec_layers_num):
    name = task+"-spec-"+ str(i)
    layer = Dense(spec_layers_units[i], activation='relu',name = name)(y)
    spec_layers[task].append(layer)
    y = Dropout(dropout[2])(layer)
  outputs.append(Dense(1, activation='sigmoid',name=task)(y))
model = Model(inputs = inputs,outputs = outputs,name = branch_term)
#print(model.summary())
#plot_model(model)

In [10]:
model.compile(optimizer = optimizer, loss = loss, metrics=metrics)

In [11]:
callback = EarlyStopping(monitor='loss', patience=3)

In [12]:
train_yk = train_y.transpose()
train_labels=[]
v_yk = validate_y.transpose()
v_labels=[]
for i in range(len(tasks)):
  train_labels.append(train_yk[i])
  v_labels.append(v_yk[i])

In [13]:
model.fit(train_x, train_labels, epochs = 50,batch_size=batch_size, verbose=0, validation_data = (validate_x, v_labels))

In [14]:
for i in range(shared_layers_num):
  name = "shared-"+str(i)
  layer = model.get_layer(name)
  if layer.trainable == True:
    layer.trainable = False
  assert layer.trainable == False
model.compile(optimizer = optimizer, loss= loss, metrics=metrics)


In [15]:
model.fit(train_x, train_labels, epochs = 50,batch_size=batch_size, verbose=0, validation_data = (validate_x, v_labels))

In [16]:
test_fn = data_fn + '/train_sets/' + branch_term + '_test.csv'
test_y = pd.read_csv(test_fn, sep='\t', header=0, index_col=0)
proteins_test = [p for p in test_y.index if p in all_data_x.index]
test_x = all_data_x.loc[proteins_test, :].values
test_y = all_data_y.loc[proteins_test, :].values.transpose()
#test_set = {}
#predict_set = {}
#test_labels = []
#for task in tasks:
#  proteins_in_test = test_y.index[test_y.loc[:, task] != inf]
#  test_y = test_y.loc[proteins_in_test, :]
#for task in tasks:
#  test_labels.append(test_y.loc[proteins_in_test,task].values)
#test_set = [test_x.loc[proteins_in_test, :].values, test_labels]


In [17]:
predict_set = model.predict(test_x)

In [18]:
test_labels = []
predict_labels = []
for i in range(len(tasks)):
  test_labels = np.concatenate((test_labels,test_y[i]))
  predict_l = predict_set[i].reshape(len(predict_set[i]))
  predict_labels = np.concatenate((predict_labels,predict_l))

i = 1
Fmax = 0
T = 0
while i < 100:
  k = i/100
  predict_labels_T = (predict_labels>k)
  f1 = f1_score(test_labels, predict_labels_T)
  
  if f1>Fmax:
    Fmax = f1
    T = i
  i = i + 1
print(str(Fmax) + "---"+ str(T))

0.06286076943999518---19
